In [9]:
import axelrod as axl
import numpy as np
import pandas as pd
from itertools import product


TURNS       = 1000         
REPS        = 1          
ALPHA       = 0.7      
N_JOBS      = -1         
EPS         = 1e-12  


In [10]:
strategies = [axl.Raider(),axl.Cooperator(), axl.GrudgerAlternator(), axl.EvolvedANNNoise05(), axl.Forgiver()]


strategy_order   = [player.__class__.__name__ for player in strategies]
strategy_mapping = {name: player.__class__ for name, player in zip(strategy_order, strategies)}

In [11]:
TURNS  = 200     
ALPHA  = 0.7

def play_pair_builtins(row_cls, col_cls):
    p_row = row_cls()
    p_col = col_cls()

    match = axl.Match([p_row, p_col], turns=TURNS)
    match.play()

    avg_row, avg_col = match.final_score_per_turn()

    coop_row = p_row.cooperations / TURNS   
    coop_col = p_col.cooperations / TURNS
    mean_coop = (coop_row + coop_col) / 2.0

    rpi = (avg_row - avg_col) / (abs(avg_row) + abs(avg_col) + 1e-9)

    smpi = ALPHA * rpi + (1 - ALPHA) * mean_coop

    return {
        "row"                     : p_row.name,
        "col"                     : p_col.name,
        "avg_score_row"           : avg_row,
        "avg_score_col"           : avg_col,
        "score_diff_row_minus_col": avg_row - avg_col,
        "row_wins_binary"         : int(avg_row > avg_col),
        "coop_rate_row"           : coop_row,
        "coop_rate_col"           : coop_col,
        "mean_coop_rate_pair"     : mean_coop,
        "RPI"                     : rpi,
        "SMPI"                     : smpi,
    }

In [12]:
pair_stats = Parallel(n_jobs=N_JOBS, verbose=10)(
    delayed(play_pair_builtins)(strategy_mapping[row], strategy_mapping[col])
    for row, col in product(strategy_order, repeat=2)
)

df_long = pd.DataFrame(pair_stats)
df_long

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.011726856231689453s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  13 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  22 out of  25 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.1s finished


,row,col,avg_score_row,avg_score_col,score_diff_row_minus_col,row_wins_binary,coop_rate_row,coop_rate_col,mean_coop_rate_pair,RPI,SMPI
0,Raider,Raider,1.660,1.660,0.000,0,0.330,0.330,0.3300,0.000000,0.099000
1,Raider,Cooperator,5.000,0.000,5.000,1,0.000,1.000,0.5000,1.000000,0.850000
2,Raider,GrudgerAlternator,2.990,0.515,2.475,1,0.005,0.500,0.2525,0.706134,0.570044
3,Raider,Evolved ANN 5 Noise 05,2.860,2.860,0.000,0,0.865,0.865,0.8650,0.000000,0.259500
4,Raider,Forgiver,1.010,1.035,-0.025,0,0.010,0.005,0.0075,-0.012225,-0.006307
5,Cooperator,Raider,0.000,5.000,-5.000,0,1.000,0.000,0.5000,-1.000000,-0.550000
6,Cooperator,Cooperator,3.000,3.000,0.000,0,1.000,1.000,1.0000,0.000000,0.300000
7,Cooperator,GrudgerAlternator,3.000,3.000,0.000,0,1.000,1.000,1.0000,0.000000,0.300000
8,Cooperator,Evolved ANN 5 Noise 05,1.755,3.830,-2.075,0,1.000,0.585,0.7925,-0.371531,-0.022322
9,Cooperator,Forgiver,3.000,3.000,0.000,0,1.000,1.000,1.0000,0.000000,0.300000


In [13]:
df_long.to_csv("../simulated_data/precalculated_metrics/smpi_data_for_SHAP.csv")